# Fine-tune Llama 3.1 8B with Unsloth
> 🗣️ [Large Language Model Course](https://github.com/mlabonne/llm-course)

In [1]:
!pip install -qqqq "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git" --progress-bar off

# We have to check which Torch version for Xformers (2.3 -> 0.0.27)
from torch import __version__; from packaging.version import Version as V
xformers = "xformers==0.0.27" if V(__version__) < V("2.4.0") else "xformers"
!pip install -qqqq --no-deps {xformers} trl peft accelerate bitsandbytes triton --progress-bar off

import torch
from trl import SFTTrainer
from datasets import load_dataset
from transformers import TrainingArguments, TextStreamer
from unsloth.chat_templates import get_chat_template
from unsloth import FastLanguageModel, is_bfloat16_supported

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.
🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


## 1. Load model for PEFT

In [3]:
# Load model
max_seq_length = 2048
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="NouRed/Med-LLaVa-QLoRA",
    trust_remote_code=True,
    max_seq_length=max_seq_length,
    load_in_4bit=True,
    dtype=None,
)

# Prepare model for PEFT
model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    lora_alpha=16,
    lora_dropout=0,
    target_modules=["q_proj", "k_proj", "v_proj", "up_proj", "down_proj", "o_proj", "gate_proj"],
    use_rslora=True,
    use_gradient_checkpointing="unsloth"
)
print(model.print_trainable_parameters())

NotImplementedError: Unsloth: llava-hf/llava-1.5-7b-hf not supported yet!
Make an issue to https://github.com/unslothai/unsloth!

## 2. Prepare data and tokenizer

In [ ]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

from datasets import load_dataset
dataset = load_dataset("yahma/alpaca-cleaned", split = "train")
#dataset = load_dataset("fresvyx/alpaca-llama2-indonesian", split = "train")
dataset = dataset.map(formatting_prompts_func, batched = True,)

## 3. Training

In [ ]:
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        # num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 300,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

In [ ]:
#@title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GiB.")
print(f"{start_gpu_memory} GiB of memory reserved.")

In [ ]:
trainer_stats = trainer.train()

In [ ]:
#@title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory         /max_memory*100, 3)
lora_percentage = round(used_memory_for_lora/max_memory*100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GiB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GiB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

In [ ]:
# prompt: In the current state, it'll generate an attributeerror due to model being none. What code (snippet) should be added to solve this error?

# Add this code snippet before trainer.train()
# if hasattr(model, "enable_gradient_checkpointing"):
#    model.enable_gradient_checkpointing()

## 4. Inference

In [ ]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        "You are an intelligent character counter", # instruction
        "How many r's are there in the word 'strawberry'? Please give the complete reasoning of your answer.", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 256, use_cache = True)
tokenizer.batch_decode(outputs)

In [ ]:
# alpaca_prompt = Copied from above
'''
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        "Bilangan manakah yang lebih besar?", # instruction
        "9,11 atau 9,9, dan sebutkan alasannya.", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 256, use_cache = True)
tokenizer.batch_decode(outputs)
'''

In [ ]:
# alpaca_prompt = Copied from above
'''
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        "Bilangan manakah yang lebih besar? Petunjuk: Kurangkan bilangan kedua dari bilangan pertama, jika hasilnya negatif, maka bilangan kedua lebih kecil daripada bilangan pertama, dan sebaliknya.", # instruction
        "9,11 atau 9,9?", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 256, use_cache = True)
tokenizer.batch_decode(outputs)
'''

In [ ]:
'''
FastLanguageModel.for_inference(model)
test_prompts = load_dataset("fka/awesome-chatgpt-prompts", split = "train")
for prompt in test_prompts["prompt"]:
  inputs = tokenizer(
  [
    alpaca_prompt.format(
        "You are a bilingual assistant who can speak both English and Indonesian fluently", # instruction
        prompt,
        "",
    )
  ], return_tensors = "pt").to("cuda")

  outputs = model.generate(**inputs, max_new_tokens = 256, use_cache = True)
  print(tokenizer.batch_decode(outputs))
'''

In [ ]:
'''
# prompt: How to change the code in the cell below so that the output is prettified like a table (i.e. pretty dataframe format instead of ugly console format)?

FastLanguageModel.for_inference(model)
test_prompts = load_dataset("fka/awesome-chatgpt-prompts", split = "train")
results = []
for prompt in test_prompts["prompt"]:
  inputs = tokenizer(
  [
    alpaca_prompt.format(
        "You are a bilingual assistant who can speak both English and Indonesian fluently. Please respond in Indonesian if and only if the prompt is in Indonesian, respond in English otherwise.", # instruction
        prompt,
        "",
    )
  ], return_tensors = "pt").to("cuda")

  outputs = model.generate(**inputs, max_new_tokens = 256, use_cache = True)
  results.append(tokenizer.batch_decode(outputs))

import pandas as pd
df = pd.DataFrame([result[0].split("\n\n###") for result in results])
df.iloc[:, :4]
'''


## 5. Save trained model

In [ ]:
df.to_csv("test_prompt_results.csv")
model.save_pretrained("lora_model") # Local saving
tokenizer.save_pretrained("lora_model")
# model.push_to_hub("your_name/lora_model", token = "...") # Online saving
# tokenizer.push_to_hub("your_name/lora_model", token = "...") # Online saving

In [ ]:
# model.save_pretrained_gguf("model", tokenizer, "q8_0")
# quant_methods = ["q2_k", "q3_k_m", "q4_k_m", "q5_k_m", "q6_k", "q8_0"]
# for quant in quant_methods:
#    model.push_to_hub_gguf("mlabonne/FineLlama-3.1-8B-GGUF", tokenizer, quant)